In [ ]:
def ring_search_large(r_in, r_out, ring_index, allowed_change, output_base_path):
    brightness_target_median = 49
    prev_circle = None
    prev_circle_radius = None
    prev_ccol = None
    prev_crow = None
    prev_movement = 0
    cross_length = 10  # Center cross
    ring_index_start = ring_index

    # Define input path directly to the cropped images folder
    output_path = Path("Results_Validation/VideoLarge/cropped_object_2")
    
    # Setting up a folder for saving results
    result_folder = f"Results_Validation/VideoLarge/cropped_object_2_Results_bw{r_in}{r_out}_{ring_index}"
    Path(result_folder).mkdir(parents=True, exist_ok=True)
    Path('ExcelsLarge').mkdir(parents=True, exist_ok=True)
    
    error_log = []
    df = pd.DataFrame(columns=['center_y(ccol)', 'center_x(crow)', 'circle_radius'])

    filenames=os.listdir(output_path)

    for filename in filenames[1942:]:
        file_path = os.path.join(output_path, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        
        brightness = image_background_brightness(image)
        background_brightness_median = image_background_brightness(image)[1]
        brightness_change = brightness_target_median - background_brightness_median
        imageBright = np.clip(image + brightness_change, 0, 255).astype('uint8')
        
        fig, axes = plt.subplots(2, 4, figsize=(15, 10))
        axes = axes.ravel()
        
        axes[0].imshow(imageBright, cmap='gray', vmin=0, vmax=255)
        axes[0].set_title('Original Image')
        
        # Fourier Transform processing
        ftimage = fft2(imageBright)
        ftimage = fftshift(ftimage)
        axes[1].imshow(20 * np.log(np.abs(ftimage) + 1), cmap='gray', vmin=0, vmax=255)
        axes[1].set_title('Fourier Transform')

        # Creating mask and filtering
        rows, cols = ftimage.shape
        crow, ccol = rows // 2, cols // 2
        mask = np.zeros((rows, cols), dtype=np.uint8)
        
        x, y = np.ogrid[:rows, :cols]
        mask_area = np.logical_and(((x - crow)**2 + (y - ccol)**2 >= r_in**2),
                                   ((x - crow)**2 + (y - ccol)**2 <= r_out**2))
        mask[mask_area] = 1
        axes[2].imshow(mask, cmap='gray')
        axes[2].set_title('Bandpass Filter')

        m_app_ftimage = ftimage * mask
        i_ftimage = ifftshift(m_app_ftimage)
        result_img = ifft2(i_ftimage)
        tmp = np.log(np.abs(result_img) + 1)
        axes[3].imshow(tmp, cmap='gray', vmin=tmp.min(), vmax=tmp.max())
        axes[3].set_title('Filtered Image')

        axes[4].plot(tmp[int(tmp.shape[0] / 2), :], color='blue')
        axes[4].set_title('Intensity Profile')
        
        crow, ccol = unravel_index(tmp.argmax(), result_img.shape)
        
        central_line_y = tmp[crow, :]
        central_line_x = tmp[:, ccol]

        smoothed_central_line_y = moving_average(central_line_y, window_size=4)
        smoothed_central_line_x = moving_average(central_line_x, window_size=4)

        peaks_y = argrelextrema(smoothed_central_line_y, np.greater)[0]
        peaks_x = argrelextrema(smoothed_central_line_x, np.greater)[0]

        if len(peaks_x) > 1 and len(peaks_y) > 1:
            ccol, x_distance = get_center_from_peaks(peaks_y, ccol)
            crow, y_distance = get_center_from_peaks(peaks_x, crow)
            circle_radius = circle_radius_fun(peaks_x, peaks_y, ring_index, ccol, crow)

            circle_radius, ring_index = circle_change_limiter(circle_radius, prev_circle_radius, allowed_change, ring_index, peaks_x, peaks_y, ccol, crow)
                
            circle_img = cv2.circle(image.copy(), (ccol, crow), int(circle_radius), (255, 0, 0), 2)
            cv2.line(circle_img, (ccol - cross_length, crow), (ccol + cross_length, crow), (255, 255, 255), 1)
            cv2.line(circle_img, (ccol, crow - cross_length), (ccol, crow + cross_length), (255, 255, 255), 1)
            axes[5].imshow(circle_img, cmap='gray', vmin=0, vmax=255)
            axes[5].set_title('Result with Circle')
        else:
            error_log.append([filename, 'Frame with no valid peaks', r_in, r_out, 2])
            axes[5].imshow(image, cmap='gray', vmin=0, vmax=255)
            axes[5].set_title('Result with Circle (No valid peaks)')

        axes[6].plot(smoothed_central_line_y, color='blue')
        for peak in peaks_y:
            axes[6].axvline(x=peak, color='red', linestyle='--')
        axes[6].set_title('Intensity Profile (Y-Axis) with Peaks')

        axes[7].plot(smoothed_central_line_x, color='green')
        for peak in peaks_x:
            axes[7].axvline(x=peak, color='red', linestyle='--')
        axes[7].set_title('Intensity Profile (X-Axis) with Peaks')

        prev_circle_radius = circle_radius
        prev_ccol = ccol
        prev_crow = crow

        new_data = {'center_y(ccol)': ccol, 'center_x(crow)': crow, 'circle_radius': circle_radius}
        df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
        df.to_excel(f"ExcelsLarge/VideoLarge_object2_2_{r_in}{r_out}_{ring_index_start}.xlsx")

        plt.tight_layout()
        plt.clf()
        plt.cla()
        plt.close('all')
        gc.collect()

    
